In [ ]:
from module import (
    evaluate_and_append,
    X,
    y,
    skf,
    results_df
)

from imblearn.pipeline import Pipeline as ImbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV


pipeline_knn = ImbPipeline([
    ('scaler', StandardScaler()),
    ('knn', KNeighborsClassifier())
])

param_grid_knn = {
    'knn__n_neighbors': [3, 5, 7, 11, 15, 21, 31, 51],
    'knn__weights': ['uniform', 'distance'],
    'knn__metric': ['euclidean', 'manhattan'],
    'knn__p': [1, 2]
}

grid_knn = GridSearchCV(
    estimator=pipeline_knn,
    param_grid=param_grid_knn,
    cv=skf,
    scoring='recall',
    n_jobs=-1,
    verbose=1,
    error_score='raise'
)

grid_knn.fit(X, y)

print('KNN best params:', grid_knn.best_params_)
print('KNN best recall:', grid_knn.best_score_)

results_df = evaluate_and_append(
    model_name='KNeighborsClassifier',
    best_estimator=grid_knn.best_estimator_,
    X=X, y=y, cv=skf,
    results_df=results_df
)

print(results_df)

Fitting 5 folds for each of 576 candidates, totalling 2880 fits
KNN best params: {'knn__metric': 'euclidean', 'knn__n_neighbors': 51, 'knn__p': 1, 'knn__weights': 'uniform', 'smote__k_neighbors': 7, 'smote__sampling_strategy': 1.0}
KNN best recall: 0.8126600284495021


                  Model  Accuracy    Recall   ROC-AUC    PR-AUC
0  KNeighborsClassifier  0.669799  0.812834  0.776909  0.111815


,Model,Accuracy,Recall,ROC-AUC,PR-AUC
0,KNeighborsClassifier,0.669799,0.812834,0.776909,0.111815
